In [1]:
import numpy as np
import pandas as pd

# Получаем данные

In [ ]:
!pip install wget

Using legacy 'setup.py install' for wget, since package 'wheel' is not installed.
    Running setup.py install for wget: started
    Running setup.py install for wget: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\konma\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
!python -m wget https://files.grouplens.org/datasets/movielens/ml-latest.zip -o ml-latest.zip


Saved under ml-latest.zip


In [ ]:
import zipfile
with zipfile.ZipFile('ml-latest.zip', 'r') as zip_ref:
    zip_ref.extractall()


### Для начала возьмем информацию из двух основных файлов movies.csv и ratings.csv

In [ ]:
from pathlib import Path
data_path = Path('ml-latest')

In [ ]:
ratings = pd.read_csv(data_path/r'ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
27753439,283228,8542,4.5,1379882795
27753440,283228,8712,4.5,1379882751
27753441,283228,34405,4.5,1379882889
27753442,283228,44761,4.5,1354159524


In [ ]:
movies = pd.read_csv(data_path/r'ml-latest/movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed)
58094,193878,Les tribulations d'une caissière (2011),Comedy
58095,193880,Her Name Was Mumu (2016),Drama
58096,193882,Flora (2017),Adventure|Drama|Horror|Sci-Fi


### Скомбинируем некоторые данные из этих таблиц в один датафрейм

In [ ]:
ratings_df = pd.merge(ratings, movies)[['userId', 'title', 'rating', 'timestamp']]
ratings_df["userId"] = ratings_df["userId"].astype(str)

### Перейдем к созданию и разделению выборки для тренировки и валидации. 
Для этого сделаем препроцессинг данных. Для каждого пользователя, выбираем последнюю по времени оценку, с учётом того, что эти пользователи уже оценили некоторое количество фильмов, превышающее определённое пороговое значение. Это даёт нам хорошее представление того процесса, который мы собираемся смоделировать. Именно этот подход нам и стоит использовать. Для этого определим функцию, которая будет получать n оценок для каждого пользователя:

In [ ]:
def get_last_n_ratings_by_user(
    df, n, min_ratings_per_user=0.5, user_colname="userId", timestamp_colname="timestamp"
):
    return (
        df.groupby(user_colname)
        .filter(lambda x: len(x) >= min_ratings_per_user)
        .sort_values(timestamp_colname)
        .groupby(user_colname)
        .tail(n)
        .sort_values(user_colname)
    )

In [ ]:
get_last_n_ratings_by_user(ratings_df, 1)

,userId,title,rating,timestamp
62738,1,Stigmata (1999),3.0,1256677500
12597906,10,Withnail & I (1987),5.0,948967730
5579877,100,"Fifth Element, The (1997)",4.5,1212324756
14156358,1000,"Grand Budapest Hotel, The (2014)",2.5,1534443939
27207753,10000,Mandy (2018),4.0,1537700357
...,...,...,...,...
15016031,99995,Englishman Who Went Up a Hill But Came Down a ...,3.0,831764418
2006885,99996,"Shawshank Redemption, The (1994)",4.0,1471744148
23947885,99997,Windtalkers (2002),4.0,1043014736
19267908,99998,"Money Pit, The (1986)",3.0,944943612


Далее с помощью этой функции определим другую функцию, которая будет помечать n оценок на пользователя так, чтобы они попадали бы в проверочную выборку. Для этого создадим отдельный столбец is_valid.

In [ ]:
def mark_last_n_ratings_as_validation_set(
    df, n, min_ratings=0.5, user_colname="userId", timestamp_colname="timestamp"
):
    df["is_valid"] = False
    df.loc[
        get_last_n_ratings_by_user(
            df,
            n,
            min_ratings,
            user_colname=user_colname,
            timestamp_colname=timestamp_colname,
        ).index,
        "is_valid",
    ] = True

    return df

In [ ]:
mark_last_n_ratings_as_validation_set(ratings_df, 1)

,userId,title,rating,timestamp,is_valid
0,1,Three Colors: Blue (Trois couleurs: Bleu) (1993),3.5,1256677221,False
1,6,Three Colors: Blue (Trois couleurs: Bleu) (1993),4.0,832059248,False
2,56,Three Colors: Blue (Trois couleurs: Bleu) (1993),4.0,1383625728,False
3,71,Three Colors: Blue (Trois couleurs: Bleu) (1993),5.0,1257795414,False
4,84,Three Colors: Blue (Trois couleurs: Bleu) (1993),3.0,999055519,False
...,...,...,...,...,...
27753439,282403,Stranglehold (1994),1.0,1524243885,False
27753440,282732,The Great Houdini (1976),3.5,1504408070,False
27753441,283000,Hotline (2014),3.5,1417317969,False
27753442,283000,Barnum! (1986),3.5,1431539331,False


In [11]:
ratings_df = pd.read_csv('/content/ratings_df (1).csv', compression='gzip')
ratings_df

,userId,title,rating,timestamp,is_valid
0,1,Three Colors: Blue (Trois couleurs: Bleu) (1993),3.5,1256677221,False
1,6,Three Colors: Blue (Trois couleurs: Bleu) (1993),4.0,832059248,False
2,56,Three Colors: Blue (Trois couleurs: Bleu) (1993),4.0,1383625728,False
3,71,Three Colors: Blue (Trois couleurs: Bleu) (1993),5.0,1257795414,False
4,84,Three Colors: Blue (Trois couleurs: Bleu) (1993),3.0,999055519,False
...,...,...,...,...,...
27753439,282403,Stranglehold (1994),1.0,1524243885,False
27753440,282732,The Great Houdini (1976),3.5,1504408070,False
27753441,283000,Hotline (2014),3.5,1417317969,False
27753442,283000,Barnum! (1986),3.5,1431539331,False


# После этого мы можем разделить данные на обучающую и тестовую выборки:

In [12]:
train_df = ratings_df[ratings_df.is_valid==False]
valid_df = ratings_df[ratings_df.is_valid==True]

# Выбор метрики
Будем оценивать с помощью кореня из средней квадратичной ошибки (Root Mean Squared Error, RMSE), т.к. он имеет тенденцию к более сильному выделению больших ошибок. Метрика будет показывать как близка потенциальная оценка пользователя фильму, который мы хотим ему порекомендовать.

# Реализация эмбеддинга
В кодирование признаков пользователей и фильмов я буду использовать матричную факторизацию со смещениемю. Для каждого пользователя и фильма создается эмбеддинг в виде матрицы. В каждой матрице пользователя каждая строка является прдпочтением отдельного пользователя. То же самое и с эмбеддингом фильма. Перемножая матрицы эмбеддингов пользователя и фильмя мы получаем матрицу оценок с какой-то погрешностью (смещением bias), которая может спрогнозировть поведение реакцию на фильмы, которые он еще не смотрел. По этому критерию и будут ранжироваться фильмы в рекомендациях.


Для начала перенумеруем id пользователя и название фильма с помощью чисел, создав свои индефикаторы. Также будем хранить таблицу для однозначного соответствия.

In [13]:
user_lookup = {v: i+1 for i, v in enumerate(ratings_df['userId'].unique())}

In [14]:
movie_lookup = {v: i+1 for i, v in enumerate(ratings_df['title'].unique())}

Используя PyTorch, создадим класс, хранящий наш DataFrame со всеми поисковыми таблицами. Также сделаем возможность достать из него оценки, выставленные пользователями фильмам с помощью метода getitem.

In [15]:
from torch.utils.data import Dataset

class UserItemRatingDataset(Dataset):
    def __init__(self, df, movie_lookup, user_lookup):
        self.df = df
        self.movie_lookup = movie_lookup
        self.user_lookup = user_lookup

    def __getitem__(self, index):
        row = self.df.iloc[index]
        user_id = self.user_lookup[row.userId]
        movie_id = self.movie_lookup[row.title]
        
        rating = torch.tensor(row.rating, dtype=torch.float32)
        
        return (user_id, movie_id), rating

    def __len__(self):
        return len(self.df)

Преопределим обучающую и валидационную выборку в контексте нашего нового класса.

In [16]:
train_dataset = UserItemRatingDataset(train_df, movie_lookup, user_lookup)
valid_dataset = UserItemRatingDataset(valid_df, movie_lookup, user_lookup)

# Создание модели
С помощью библиотеки PyTorch создадим модель, которая будет содержать слой эмбеддинга.При указании размера слоя эмбеддинга необходимо сделать так, чтобы в нём присутствовали бы все значения, которые могут встретиться в процессе обучения и проверки модели. Из-за этого мы используем количество уникальных элементов, имеющихся в полном наборе данных, а не только в учебном наборе.

In [17]:
import torch
from torch import nn

class MfDotBias(nn.Module):

    def __init__(
        self, n_factors, n_users, n_items, ratings_range=None, use_biases=True
    ):
        super().__init__()
        self.bias = use_biases
        self.y_range = ratings_range
        self.user_embedding = nn.Embedding(n_users+1, n_factors, padding_idx=0)
        self.item_embedding = nn.Embedding(n_items+1, n_factors, padding_idx=0)

        if use_biases:
            self.user_bias = nn.Embedding(n_users+1, 1, padding_idx=0)
            self.item_bias = nn.Embedding(n_items+1, 1, padding_idx=0)

    def forward(self, inputs):
        users, items = inputs
        dot = self.user_embedding(users) * self.item_embedding(items)
        result = dot.sum(1)
        if self.bias:
            result = (
                result + self.user_bias(users).squeeze() + self.item_bias(items).squeeze()
            )

        if self.y_range is None:
            return result
        else:
            return (
                torch.sigmoid(result) * (self.y_range[1] - self.y_range[0])
                + self.y_range[0]
            )

# Создание callback для отслеживания метрики

In [18]:
!pip install pytorch_accelerated

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0


In [19]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 33.8 MB/s eta 0:00:00


In [20]:
from functools import partial

from pytorch_accelerated import Trainer, notebook_launcher 
from pytorch_accelerated.trainer import TrainerPlaceholderValues, DEFAULT_CALLBACKS
from pytorch_accelerated.callbacks import EarlyStoppingCallback, SaveBestModelCallback, TrainerCallback, StopTrainingError
import torchmetrics

INFO:pytorch_accelerated:Setting random seeds


In [21]:
class RecommenderMetricsCallback(TrainerCallback):
    def __init__(self):
        self.metrics = torchmetrics.MetricCollection(
            {
                "mse": torchmetrics.MeanSquaredError(),
                "mae": torchmetrics.MeanAbsoluteError(),
            }
        )

    def _move_to_device(self, trainer):
        self.metrics.to(trainer.device)

    def on_training_run_start(self, trainer, **kwargs):
        self._move_to_device(trainer)

    def on_evaluation_run_start(self, trainer, **kwargs):
        self._move_to_device(trainer)

    def on_eval_step_end(self, trainer, batch, batch_output, **kwargs):
        preds = batch_output["model_outputs"]
        self.metrics.update(preds, batch[1])

    def on_eval_epoch_end(self, trainer, **kwargs):
        metrics = self.metrics.compute()
        
        mse = metrics["mse"].cpu()
        trainer.run_history.update_metric("mae", metrics["mae"].cpu())
        trainer.run_history.update_metric("mse", mse)
        trainer.run_history.update_metric("rmse",  math.sqrt(mse))

        self.metrics.reset()

# Обучение модели
В качестве функции потерь мы выбрали MSE, в качестве оптимизатора — AdamW, скорость обучения задаём с помощью OneCycle. Это приводит нас к такой обучающей функции:

In [22]:
def train_mf_model():
    model = MfDotBias(
        120, len(user_lookup), len(movie_lookup), ratings_range=[0, 5.5]
    )
    loss_func = torch.nn.MSELoss()

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)

    create_sched_fn = partial(
        torch.optim.lr_scheduler.OneCycleLR,
        max_lr=0.01,
        epochs=TrainerPlaceholderValues.NUM_EPOCHS,
        steps_per_epoch=TrainerPlaceholderValues.NUM_UPDATE_STEPS_PER_EPOCH,
    )

    trainer = Trainer(
        model=model,
        loss_func=loss_func,
        optimizer=optimizer,
        callbacks=(
            RecommenderMetricsCallback,
            *DEFAULT_CALLBACKS,
            SaveBestModelCallback(watch_metric="mae"),
            EarlyStoppingCallback(
                early_stopping_patience=2,
                early_stopping_threshold=0.001,
                watch_metric="mae",
            ),
        ),
    )

    trainer.train(
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        num_epochs=2,
        per_device_batch_size=512,
        create_scheduler_fn=create_sched_fn,
    )

### Теперь можно запустить обучение, передав эту функцию функции notebook_launcher:

In [23]:
torch.cuda.is_available()

True

In [24]:
notebook_launcher(train_mf_model, num_processes=1)

Launching training on one GPU.

Starting training run

Starting epoch 1


100%|██████████| 53653/53653 [51:47<00:00, 17.26it/s]



train_loss_epoch: 3.676194190979004


100%|█████████▉| 552/554 [00:28<00:00, 22.01it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/launchers.py:103 in notebook_launcher         │
│                                                                                                  │
│   100 │   │   │   print("Launching training on one GPU.")                                        │
│   101 │   │   else:                                                                              │
│   102 │   │   │   print("Launching training on one CPU.")                                        │
│ ❱ 103 │   │   function(*args)                                                                    │
│   104 │   else:                                                                                  │
│   105 │   │   if num_processes is None:                                                          │
│   106 │   │   │   raise ValueError(                                                              │
│ in train_mf_model:32                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pytorch_accelerated/trainer.py:468 in train              │
│                                                                                                  │
│    465 │   │   if self.create_scheduler_fn is not None:                                          │
│    466 │   │   │   self.scheduler = self.create_scheduler()                                      │
│    467 │   │                                                                                     │
│ ❱  468 │   │   self._run_training()                                                              │
│    469 │                                                                                         │
│    470 │   def evaluate(                                                                         │
│    471 │   │   self,                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pytorch_accelerated/trainer.py:699 in _run_training      │
│                                                                                                  │
│    696 │   │   │   try:                                                                          │
│    697 │   │   │   │   self._run_train_epoch(self._train_dataloader)                             │
│    698 │   │   │   │   if self.eval_dataset is not None:                                         │
│ ❱  699 │   │   │   │   │   self._run_eval_epoch(self._eval_dataloader)                           │
│    700 │   │   │   │   self.training_run_epoch_end()                                             │
│    701 │   │   │   │   self.callback_handler.call_event(                                         │
│    702 │   │   │   │   │   "on_training_run_epoch_end",                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pytorch_accelerated/trainer.py:884 in _run_eval_epoch    │
│                                                                                                  │
│    881 │   │   metric_name = "eval_loss_epoch" if is_training else "evaluation_loss"             │
│    882 │   │   self._add_epoch_loss_to_run_history(metric_name)                                  │
│    883 │   │                                                                                     │
│ ❱  884 │   │   self.callback_handler.call_event(           

# Option

# Сравнение с базовой моделью
Для построения базовой модели, можем просто прогнозировать среднюю оценку по каждому фильму, не обращая внимания на контекст. Так как среднее значение может подвергаться значительному влиянию показателей, сильно отличающихся от обычных, используем для построения модели медиану значений.
Затем можно воспользоваться этим показателем в виде прогноза для каждой оценки в проверочной выборке и посчитать показатели качества модели.
Затем сравнить их с нашей обученной моделью и сделать выводы об качестве.

In [2]:
import math
from sklearn.metrics import mean_squared_error

median_rating = train_df.rating.median(); median_rating
predictions = np.array([median_rating]* len(valid_df))

mae = mean_absolute_error(valid_df.rating, predictions)
mse = mean_squared_error(valid_df.rating, predictions)
rmse = math.sqrt(mse)

print(f'rmse: {rmse}')